In [1]:
import pandas as pd

In [2]:
# File1 should probably be the CD_CSV file always
def main(file1, file2):
    f1 = pd.read_csv(file1, sep='\t', encoding='utf-8')
    df_file1 = pd.DataFrame(f1)
    df_file2 = pd.DataFrame(pd.read_csv(file2, sep='\t', encoding='utf-8'))

    # print(df_file1['County'].head(5))
    # By making this new DataFrame I cut out the excess index column from the second file.
    stats_from_file_two = pd.DataFrame(data=[df_file2['County'], df_file2['Higher Degree'], df_file2['H.S Diploma'], df_file2['No H.S Diploma']]).transpose()

    df_output = pd.merge(df_file1, stats_from_file_two, on='County')
    # print(df_output.head(5))

    df_output.to_csv('CD_and_Statistical_Atlas_Data_Combined', sep='\t', encoding='utf-8')

In [ ]:
if __name__ == '__main__':
    main('Fully_Cleaned_CD_Data', 'Fully_Cleaned_AEA_Data')